 **추론 모델 훈련 방법은 [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) 참고**


### Installation

In [ ]:
# ! pip install unsloth vllm
# # Gemma3를 사용할 경우
# ! pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

### Unsloth

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 512

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    load_in_8bit = False,
    full_finetuning = False,
    # token = "hf_...",
)

### LoRA

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

### 데이터 준비
[GSM8K](https://huggingface.co/datasets/openai/gsm8k) 데이터셋 사용

In [ ]:
from datasets import load_dataset
dataset = load_dataset("openai/gsm8k", "main", split = "train")

In [ ]:
dataset

In [ ]:
dataset[0]["question"]

In [ ]:
dataset[0]["answer"]

모든 answer에 ####가 있음으로 이를 기준으로 최종 정답 추출

In [ ]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

사용자 정의가 가능한 시스템 프롬프트를 생성. 작업 또는 사고/추론 섹션을 위한 4개의 추가 기호와 최종 답변을 추가

In [ ]:
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
# 한국어로 할 경우(다만 gemma3-1b 모델에 경우 영어만 지원)
# 문제가 주어집니다.
# 문제에 대해 생각하고, 풀이 과정을 제공해주세요.
# 풀이 과정은 {reasoning_start}와 {reasoning_end} 사이에 작성해주세요.
# 그 후, 해결책을 {solution_start}와 {solution_end} 사이에 제공해주세요.

In [ ]:
system_prompt

데이터셋 매핑

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})

In [ ]:
dataset[0]

추론 섹션과 답변에 맞는 정규식 형식을 생성

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

형식에 맞는 보상 함수를 구현, 성공하면 3점을 보상으로 준다.

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

실패할 경우, 부분적으로 형식을 따르는 모델에 보상을 준다.

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # 얼마나 많은 키워드가 표시되는지 세어보세요. 너무 많으면 페널티가 적용됩니다!
        # 1개가 보이면 플러스 포인트!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

마지막으로, 생성된 답을 추출하고 보상하거나 처벌한다. 또한 비율을 통해 정답과 정답의 근접성에 따라 보상한다.

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # 정답은 3점을 얻습니다!
        if guess == true_answer:
            score += 3.0
        # 공백이 있는 경우에도 매칭되도록 합니다.
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # 답이 일정 부분 유사한 경우에도 보상을 줍니다.
            # 즉, 답변이 어느 정도 범위 내에 있으면 보상합니다!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # 틀린 답변에 대한 처벌
            except:
                score -= 0.5
        scores.append(score)
    return scores

때로는 답이 하나의 숫자가 아닌 문장일 수도 있습니다. 예를 들어, "The solution is $20"에서 20을 추출합니다.

In [ ]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

In [ ]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # 숫자로 변환
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

### Train the model

In [ ]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 4, # 메모리 사용량이 많다면 줄이세요.
    gradient_accumulation_steps = 4, # 메모리 사용량이 많다면 늘리세요.
    num_generations = 2, # 중요! 몇 개의 답변을 생성할 것인지 결정. 메모리 사용량이 많다면 줄이세요.
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # 전체 학습을 실행하려면 1이상으로 설정하세요.
    max_steps = 200, # 테스트 시에만 사용
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "wandb", # Weights & Biases, 사용하지 않을 시 "none"
    run_name="gemma3_1B_GRPO", # Wanb 사용시에만
    output_dir = "outputs", # checkpoint 저장 경로
)

목표는 reward열이 증가하는 것입니다!
150~200 step을 기다려야 할 수도 있습니다. 처음 100step은 보상이 없을 가능성이 큽니다.

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)

In [ ]:
trainer.train()

<a name="Inference"></a>
### Inference

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # 생성을 위해 반드시 True
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # 더 긴 출력을 원하시면 늘리세요!
    # Gemma-3 추천 설정값!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

### Saving to float16 for VLLM

배포를 위해 직접 저장하는 것도 지원합니다 float16! 폴더에 저장합니다 gemma-3-finetune. 실행되도록 `if False` 설정 하세요! `if True`

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

Hugging Face 계정에 업로드/푸시하려면, Hugging Face 토큰과 업로드 위치를 설정하고 추가하세요 `if False`!`if True`

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
GGUF/ 에 저장하려면 llama.cpp이제 모든 모델에서 기본적으로 지원합니다! 지금은 4비트에 대한 Q8_0, F16 or BF16정밀도로 쉽게 변환할 수 있습니다.

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

마찬가지로, Hugging Face 계정으로 GGUF를 푸시하려면, Hugging Face 토큰을 설정 if False하고 if True위치를 업로드하세요!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )